In [1]:
# https://www.kaggle.com/code/ryanholbrook/hybrid-models

## Introduction

Linear regression excels at extrapolating trends, but can't learn interations. XGBoost excels at learning interactions, but can't extrapolate trends. In this lesson, we'll learn how to create "hybrid" forecasters that combine complementary learning algorithms and let the strengths of one make up for the weakness of the other.

## Components and Residuals

